In [1]:
import numpy as np
from scipy.sparse import csr_matrix

In [2]:
wv_exp = np.linspace(300, 322, num=5000)
wv_lg = np.linspace(302, 320, num=500)

In [3]:
delta_wv = wv_exp[np.newaxis].transpose() - wv_lg
delta_wv

array([[ -2.        ,  -2.03607214,  -2.07214429, ..., -19.92785571,
        -19.96392786, -20.        ],
       [ -1.99559912,  -2.03167126,  -2.06774341, ..., -19.92345483,
        -19.95952698, -19.99559912],
       [ -1.99119824,  -2.02727038,  -2.06334253, ..., -19.91905395,
        -19.9551261 , -19.99119824],
       ...,
       [ 19.99119824,  19.9551261 ,  19.91905395, ...,   2.06334253,
          2.02727038,   1.99119824],
       [ 19.99559912,  19.95952698,  19.92345483, ...,   2.06774341,
          2.03167126,   1.99559912],
       [ 20.        ,  19.96392786,  19.92785571, ...,   2.07214429,
          2.03607214,   2.        ]])

In [4]:
fwhm = 0.1
threshold = 3
delta_wv[delta_wv<-fwhm*threshold] = 0
delta_wv[delta_wv> fwhm*threshold] = 0
delta_wv

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [5]:
delta_wv = csr_matrix(delta_wv)
delta_wv

<5000x500 sparse matrix of type '<class 'numpy.float64'>'
	with 68164 stored elements in Compressed Sparse Row format>

In [6]:
%timeit delta_wv.expm1()

76.2 µs ± 6.5 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [7]:
delta_wv_array = delta_wv.toarray()
%timeit np.exp(delta_wv_array)

4.47 ms ± 93.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
%timeit np.exp(delta_wv.data)

22.5 µs ± 2.25 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [14]:
_where = np.logical_and(-fwhm*threshold<delta_wv_array, delta_wv_array<fwhm*threshold)
a = np.zeros_like(delta_wv_array)
%timeit np.exp(delta_wv_array, out=a, where=_where)

2.96 ms ± 57.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


**用where的方法效率很低，转换成稀疏矩阵**

In [15]:
def func_where():
    _where = np.logical_and(-fwhm*threshold<delta_wv_array, delta_wv_array<fwhm*threshold)
    a = np.zeros_like(delta_wv_array)
    np.exp(delta_wv_array, out=a, where=_where)
%timeit func_where()

17.6 ms ± 59 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [31]:
_array = delta_wv.toarray()
def func_sparse(_array):
    _array[_array<-fwhm*threshold] = 0
    _array[_array> fwhm*threshold] = 0
    _array = csr_matrix(_array)
    np.exp(_array.data)
%timeit func_sparse(_array)

22.6 ms ± 893 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [34]:
delta_wv.data = delta_wv.data + 1

In [ ]:
de